In [8]:
import os
import gzip
from Bio import SeqIO
import pandas as pd

# Paths
dna_dir = "data/dna_sequences/"
gff3_dir = "data/gff3_files/"

# Collect all FASTA and GFF3 files
fasta_files = sorted([
    os.path.join(dna_dir, f) for f in os.listdir(dna_dir)
    if f.lower().endswith(".fa.gz") or f.lower().endswith(".fasta")
])
gff3_files = sorted([
    os.path.join(gff3_dir, f) for f in os.listdir(gff3_dir)
    if f.lower().endswith(".gff3")
])

# Function to parse GFF3 and extract genes
def parse_gff3(gff3_file):
    genes = []
    with open(gff3_file, encoding="utf-8") as f:
        for line in f:
            if line.startswith("#"):
                continue
            parts = line.strip().split("\t")
            if len(parts) < 9:
                continue
            if parts[2] == "gene":
                start = int(parts[3]) - 1  # GFF is 1-based
                end = int(parts[4])
                strand = parts[6]
                attr = parts[8]
                gene_id = "NA"
                for field in attr.split(";"):
                    if field.startswith("ID="):
                        gene_id = field.split("=")[1]
                        break
                genes.append((start, end, strand, gene_id))
    return genes

# Extract gene sequences
gene_sequences = []

for fasta_path, gff_path in zip(fasta_files, gff3_files):
    print(f"Processing: {os.path.basename(fasta_path)} with {os.path.basename(gff_path)}")

    # Read the chromosome sequence (handle gzipped or plain files)
    if fasta_path.endswith(".gz"):
        with gzip.open(fasta_path, "rt", encoding="utf-8") as f:
            record = next(SeqIO.parse(f, "fasta"))
    else:
        with open(fasta_path, "r", encoding="utf-8") as f:
            record = next(SeqIO.parse(f, "fasta"))
    chrom_seq = record.seq

    # Parse genes from GFF3
    genes = parse_gff3(gff_path)

    # Extract gene sequences
    for start, end, strand, gene_id in genes:
        gene_seq = chrom_seq[start:end]
        if strand == "-":
            gene_seq = gene_seq.reverse_complement()
        gene_sequences.append({
            "gene_id": gene_id,
            "chrom": record.id,
            "start": start,
            "end": end,
            "strand": strand,
            "sequence": str(gene_seq)
        })

# Convert to DataFrame
df_genes = pd.DataFrame(gene_sequences)

# Preview
print(df_genes.head())


Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.1.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.1.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.10.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.10.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.2.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.2.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.3.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.3.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.4.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.4.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.5.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.5.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.6.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.6.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.7.fa.gz with Zea_mays.Zm-B73-REFER

In [9]:
import pandas as pd

# Define the window size around TSS (±10,000 bp)
WINDOW_SIZE = 10000

# Calculate TSS and window region
def compute_tss_window(df_genes):
    windows = []
    for _, row in df_genes.iterrows():
        if row["strand"] == "+":
            tss = row["start"]
        else:
            tss = row["end"]
        window_start = max(0, tss - WINDOW_SIZE)
        window_end = tss + WINDOW_SIZE
        windows.append({
            "gene_id": row["gene_id"],
            "chrom": row["chrom"],
            "start": window_start,
            "end": window_end,
            "strand": row["strand"],
            "tss": tss
        })
    return pd.DataFrame(windows)

# Use this on your existing df_genes
df_windows = compute_tss_window(df_genes)
print(df_windows.head())


                gene_id chrom   start     end strand     tss
0  gene:Zm00001eb000010     1   24616   44616      +   34616
1  gene:Zm00001eb000020     1   36762   56762      -   46762
2  gene:Zm00001eb000030     1   96620  116620      -  106620
3  gene:Zm00001eb000040     1   98196  118196      -  108196
4  gene:Zm00001eb000050     1  104382  124382      -  114382


In [10]:
from Bio import SeqIO
from Bio.Seq import Seq
import gzip

# Make a dictionary for quick lookup by chromosome
fasta_sequences = {}

for fasta_path in fasta_files:
    chrom_name = os.path.basename(fasta_path).split("chromosome.")[1].split(".")[0]
    if fasta_path.endswith(".gz"):
        with gzip.open(fasta_path, "rt") as handle:
            record = next(SeqIO.parse(handle, "fasta"))
    else:
        with open(fasta_path, "r") as handle:
            record = next(SeqIO.parse(handle, "fasta"))
    fasta_sequences[chrom_name] = record.seq

# Extract window sequences
def extract_window_sequence(row):
    chrom_seq = fasta_sequences[str(row["chrom"])]
    seq = chrom_seq[row["start"]:row["end"]]
    return str(seq.reverse_complement() if row["strand"] == "-" else seq)

df_windows["sequence"] = df_windows.apply(extract_window_sequence, axis=1)


In [11]:
import numpy as np

def one_hot_encode(sequence):
    mapping = {"A": 0, "C": 1, "G": 2, "T": 3}
    array = np.zeros((4, 20000), dtype=np.float32)
    for i, base in enumerate(sequence.upper()):
        if i >= 20000:
            break
        if base in mapping:
            array[mapping[base], i] = 1.0
    return array

# Encode all and store as list
encoded_sequences = [one_hot_encode(seq) for seq in df_windows["sequence"]]


In [25]:
from tensorflow.keras.models import load_model

model_path = "EPInformer/pretrained_enhancer_encoder/fold_1_best_GM12878_seq2activity.h5"
epinformer_model = load_model(model_path)


TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
import numpy as np

# Input shape expected: (batch_size, 4, 20000)
input_data = np.array(encoded_sequences)

# If needed, reshape to match the model's input
input_data = np.transpose(input_data, (0, 2, 1))  # (batch, 20000, 4)

predictions = epinformer_model.predict(input_data)


In [24]:
pip install numpy==1.24.4

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Dell\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.
